CREATE DATASET sur un corpus de test  sur le corpus téléchargé de projet guttenberg.J'ai d'abord installé le librarie nltk qui contiens le corpus gutenberg ainsi que le stopwords etc. 

IMPORT PACKAGES

In [1]:
from nltk import tokenize
import numpy as np
import random
import pandas as pd

LOAD DATA AND CREATE COMBINED DATASET

In creating the sentence lists, we exclude sentences of less than 5 characters in length, as these are unlikely to be proper sentences and are likely too short to contain any useful information. As the sentence tokenizer has difficulties in identifying the end of sentences under some circumstances (e.g. if the full-stop at the end of the sentence is contained within quotation marks), we make some minor adjustments to the text prior to tokenization using the replace function.

In [2]:
def split_text(filepath, min_char):
    """Convert text file to a list of sentences.
    
    Args:
    filepath: string. Filepath of text file.
    min_char: int. Minimum number of characters required for a sentence to be
    included.

    Returns:
    sentences: list of strings. List of sentences containined in the text file.
    """
    # Load data into string variable and remove new line characters
    file = open(filepath, "r", encoding="utf8")
    text = file.read().replace('\n', ' ')
    text = text.replace('.”', '”.').replace('."', '".').replace('?”', '”?').replace('!”', '”!')
    text = text.replace('--', ' ').replace('. . .', '').replace('_', '')
    file.close()
    
    # Split text into a list of sentences
    sentences = tokenize.sent_tokenize(text)
    
    # Remove sentences that are less than min_char long
    sentences = [sent for sent in sentences if len(sent) >= min_char]

    return list(sentences)

CREATE SENTENCE LIST FOR EACH AUTHOR

In [18]:
# Set parameter values
min_char = 5

# Create lists
frankenstein = split_text('Books/frankenstein.txt', min_char = min_char)
lastMan = split_text('Books/LastMan.txt', min_char = min_char)

books = frankenstein + lastMan

In [52]:
# Print length of each list

text_dict = {'Frankenstein': frankenstein, 'Last Man': lastMan }


for key in text_dict.keys():
    print(key, ':', len(text_dict[key]))
    

Frankenstein : 3266
Last Man : 6938


All lists contain between 8641 and 14414 sentences. So that our final dataset doesn't become skewed towards a single author, we will randomly select 8500 sentences from each list (without replacement) to form the final dataset.

Select and combine sentences

In [53]:
# Set random seed
np.random.seed(1)

# Set length parameter
max_len = 500

# Select sentences
names = [frankenstein,lastMan]
combined = []

for name in names:
    name = np.random.choice(name, max_len, replace = False)
    combined += list(name)

print('The length of the combined list is:', len(combined))

The length of the combined list is: 1000


Create labels list

In [58]:
labels = ['Frankenstein']*max_len + ['LastMan']*max_len

print('The length of the labels list is:', len(labels))

The length of the labels list is: 1000


Randomly sort data

We randomly shuffle the data to avoid any issues arising from the bunching together of sentences by a single author.

In [59]:
# Set random seed
random.seed(3)

# Randomly shuffle data
zipped = list(zip(combined, labels,))
random.shuffle(zipped)
combined, labels  = zip(*zipped)


Create and export final dataset

In [61]:
# Create pandas dataframe
out_data = pd.DataFrame()
out_data['text'] = combined
out_data['book'] = labels

print(out_data.head())

                                                text          book
0  Our situation was somewhat dangerous, especial...  Frankenstein
1                        “Get well—and return to us.       LastMan
2  I had hitherto supposed him to be the murderer...  Frankenstein
3  Human being there was none to reply; and the i...  Frankenstein
4  If for one instant I had thought what might be...       LastMan


In [62]:
# Export as a csv file
out_data.to_csv('shelley_data.csv', index=False)